## 1. Set Up Environment

In [3]:
import pyodbc
import sys
from datetime import date
from selenium import webdriver
from sqlalchemy import create_engine
from expedia_scrape import expedia_flight
from kayak_scrape import kayak_flight
from sql_to_excel_v2 import sql_excel

## 2. Call The Scrapers And Place Data In SQL

In [4]:
### Set Up Arguments For All Functions Used ################################################################################ 

args = ['roundtrip', 'business class', 'lhr - london', 'akj - asahikawa', date(2024,3,2), date(2024,3,22), False]


### Set Up ChromeDriver ####################################################################################################

# Create ChromeOptions instance
chrome_options = webdriver.ChromeOptions()

# Adding argument to disable the AutomationControlled flag
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# Adding argument for incognito mode
chrome_options.add_argument("--incognito")

# Exclude the collection of enable-automation switches
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Turn-off userAutomationExtension
chrome_options.add_experimental_option("useAutomationExtension", False) 

chrome_driver = "C:/ChromeDrive/chromedriver"


### Set Up ODBC Connection #################################################################################################

db = []
with open('ODBC_Conn.txt') as file:
    for line in file:
        db.append(line.rstrip())

conn = pyodbc.connect(f'DRIVER={db[2]};\
                        SERVER={db[0]};\
                        DATABASE={db[1]};\
                        UID={db[3]};PWD={db[4]}')


### Run all Scrapers And Place Outputs Into SQL Tables If All Scrapers Ran Successfully ####################################
    
f1 = expedia_flight(chrome_driver, chrome_options, conn)

ticket1 = f1.get_ticket_info(type_=args[0], class_=args[1], leave=args[2], going=args[3],
                        depart=args[4], return_=args[5], nonstop=args[6],
                        trav=[1,0,0,0])

if len(ticket1[0]) > 1:
        pass
else:
    sys.exit("Flight ticket info is empty")

f2 = kayak_flight(chrome_driver, chrome_options, conn)

ticket2 = f2.get_ticket_info(type_=args[0], class_=args[1], leave=args[2], going=args[3],
                        depart=args[4], return_=args[5], nonstop=args[6],
                        trav=[1,0,0,0,0,0], bags=[1,0])

if len(ticket2[0]) > 1:
        pass
else:
    sys.exit("Flight ticket info is empty")

f1.info_to_sql(ticket1)
f2.info_to_sql(ticket2)


### Pull the sql tables, clean them, and place them into an Excel file #####################################################
engine = create_engine(f"mssql+pyodbc://{db[3]}:{db[4]}@{db[0]}/{db[1]}?driver={db[2]}")
conn = engine.connect()
sql_excel(conn)